# Compare my code with zfturbo 

In [1]:
import datetime
import os
from collections import defaultdict
import operator

In [2]:
from datetime import datetime
import logging
logging.basicConfig(level=logging.DEBUG)

from operator import itemgetter

from copy import deepcopy
import numpy as np
from zfturbo_script_mass_hashes_personal_recommendations import read_data, get_profiles, personal_recommendations_to_proba, common_recommendations_to_proba, get_target_labels, predict_score

In [3]:
DATA_PATH = "../data"

## Small test

ZFTurbo

In [4]:
from zfturbo_script_mass_hashes import get_hashes as zf_get_hashes
from zfturbo_script_mass_hashes import sort_main_arrays as zf_sort_main_arrays
from zfturbo_script_mass_hashes import add_data_to_main_arrays as zf_add_data_to_main_arrays

Current set of hash indexes: [(7, 8, 14, 19, 23), (4, 5, 7, 9, 16)]


In [5]:
from zfturbo_script_mass_hashes import get_predictions as zf_get_predictions
from zfturbo_script_mass_hashes import get_real_values as zf_get_real_values
from zfturbo_script_mass_hashes import apk as zf_apk

In [6]:
f = open(os.path.join(DATA_PATH, "train_ver2.csv"), 'r')
first_line = f.readline().strip()
# Remove " symbols in the header. Header is like : "a","b","c"...
first_line = first_line.replace("\"", "")
# map names <-> target labels
map_names = first_line.split(",")[24:]

In [7]:
# Normal variables
customer = dict()
best = defaultdict(lambda: defaultdict(int))
overallbest = defaultdict(int)

# Validation variables
customer_valid = dict()
best_valid = defaultdict(lambda: defaultdict(int))
overallbest_valid = defaultdict(int)

valid_part = []

In [8]:
# Calc counts
total = 0
while 1:
    line = f.readline()[:-1]
    total += 1

    if line == '':
        break

    # Parse line : 
    if "\"" in line:
        tmp1 = line.split("\"")
        arr = tmp1[0][:-1].split(",") + [tmp1[1]] + tmp1[2][1:].split(',')
    else:
        arr = line.split(",")
    arr = [a.strip() for a in arr]

    # Normal part
    zf_add_data_to_main_arrays(arr, best, overallbest, customer)

    
    # Valid part
    if '2015-02' not in arr[0]:
        zf_add_data_to_main_arrays(arr, best_valid, overallbest_valid, customer_valid)
    else:
        valid_part.append(arr)

    if total % 100000 == 0:
        print('Process {} lines | current month : {}'.format(total, arr[0]))
#         break
        
    if '2015-03' in arr[0]:
        break
        
print "TOTAL : ", total

Process 100000 lines | current month : 2015-01-28
Process 200000 lines | current month : 2015-01-28
Process 300000 lines | current month : 2015-01-28
Process 400000 lines | current month : 2015-01-28
Process 500000 lines | current month : 2015-01-28
Process 600000 lines | current month : 2015-01-28
Process 700000 lines | current month : 2015-02-28
Process 800000 lines | current month : 2015-02-28
Process 900000 lines | current month : 2015-02-28
Process 1000000 lines | current month : 2015-02-28
Process 1100000 lines | current month : 2015-02-28
Process 1200000 lines | current month : 2015-02-28
TOTAL :  1252852


In [9]:
f.close()

In [10]:
print('Sort best arrays...')
print('Hashes num: ', len(best))
print('Hashes valid num: ', len(best_valid))
print('Valid part: ', len(valid_part))

Sort best arrays...
('Hashes num: ', 734434)
('Hashes valid num: ', 727939)
('Valid part: ', 627394)


In [11]:
# Normal
best, overallbest = zf_sort_main_arrays(best, overallbest)

# Valid
best_valid, overallbest_valid = zf_sort_main_arrays(best_valid, overallbest_valid)

In [ ]:
map7 = 0.0
print('Validation...')
count = -1
for i, arr1 in enumerate(valid_part):
    
    predicted = zf_get_predictions(arr1, best_valid, overallbest_valid, customer_valid)
    real = zf_get_real_values(arr1, customer_valid)
    score = zf_apk(real, predicted)
    #print "-- i : ", i, arr1[1], " score : ", score, " | predicted : ", predicted, ", real : ", real
    map7 += score
    
    count-= 1
    if count == 0:
        break

if len(valid_part) > 0:
    map7 /= len(valid_part)
print('Predicted score: {}'.format(map7))

Validation...


My code

In [ ]:
train_filename = os.path.join(DATA_PATH, "train_ver2.csv")

In [ ]:
import math
from common import parse_line, clean_data, process_data, get_choices, get_user
from zfturbo_script_mass_hashes_personal_recommendations import update_common_recommendations, update_personal_recommendations, update_product_stats, get_last_choice, to_yearmonth

In [ ]:
# def update_common_recommendations(common_recommendations, row, get_profiles_func):
#     profiles = get_profiles_func(row)
#     choices = get_choices(row)
#     for profile in profiles:
#         # Update common recommendations
#         for i, t in enumerate(choices):
#             if t == "1":
#                 common_recommendations[profile][i] += 1
#         common_recommendations[profile]['total'] += 1


def read_data(reader, yearmonth_begin, nb_months, get_profiles_func, return_raw_data=False,
              personal_recommendations=None, common_recommendations=None, product_stats=None):
    """
    :param reader:
    :param yearmonth_begin: e.g. 201501
    :param nb_months: e.g. 5 -> 201501, 201502, 201503, 201504, 201505
    :return:
    (personal_recommendations,
     common_recommendations,
     product_stats)

     or if return_raw_data == True:

     (personal_recommendations,
     common_recommendations,
     product_stats,
     raw_data)

     raw_data : list of read and parsed lines

    """

    def _to_yearmonth_str(yearmonth):
        year = int(math.floor(yearmonth * 0.01))
        month = yearmonth - year * 100
        return "%s-%s" % (str(year), str(month).zfill(2))

    year = int(math.floor(yearmonth_begin * 0.01)) * 100
    dates = [_to_yearmonth_str(yearmonth_begin), ]
    current_yearmonth = yearmonth_begin
    for i in range(1, nb_months):
        current_yearmonth += 1
        if current_yearmonth - year > 12:
            year += 100
            current_yearmonth = year + 1
        dates.append(_to_yearmonth_str(current_yearmonth))

    logging.info("- READ DATA : months to read {}".format(dates))

    raw_data = []

    if common_recommendations is None:
        common_recommendations = defaultdict(lambda: defaultdict(float))
    if personal_recommendations is None:
        personal_recommendations = defaultdict(lambda: defaultdict(float))
    if product_stats is None:
        product_stats = defaultdict(int)

    # Loop on lines in the file reader:
    removed_rows = 0
    total = 0
    while True:
        line = reader.readline()[:-1]
        total += 1

        if line == '':
            break
        row = parse_line(line)

        # data : ['2015-01-28',
        #  user id -> '1375586',
        #  profile -> 'N', 'ES', 'H', '35', '2015-01-12', '0', '6', '1', '', '1.0', 'A', 'S', 'N', '', 'KHL', 'N', '1', '29', 'MALAGA', '1', '87218.1', '02 - PARTICULARES',
        #  choices -> '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']

        yearmonth_str = row[0][:7]
        if yearmonth_str not in dates:
            yearmonth = to_yearmonth(yearmonth_str)
            if yearmonth - yearmonth_begin < 0:
                total = 0
                continue
            else:
                break

        row = clean_data(row)

        if len(row) == 0:
            removed_rows += 1
            continue

        if return_raw_data:
            raw_data.append(row)

        processed_row = process_data(row)
        #processed_row = row

        last_choice = get_last_choice(get_user(processed_row), personal_recommendations)
        update_common_recommendations(common_recommendations, processed_row, get_profiles_func, last_choice)
        update_product_stats(product_stats, row, last_choice)
        update_personal_recommendations(personal_recommendations, processed_row)

        if total % 100000 == 0:
            logging.info('-- Processed {} lines . Current month : {}'.format(total, yearmonth_str))
    
    logging.debug("-- Removed rows : %s" % removed_rows)
    print "TOTAL : ", total
    
    if return_raw_data:
        return personal_recommendations, common_recommendations, product_stats, raw_data
    return personal_recommendations, common_recommendations, product_stats


In [ ]:
logging.info('--- Run solution ---')
reader = open(train_filename, "r")
target_labels = get_target_labels(reader.readline())

nb_months_validation = 1

(personal_recommendations_validation,
 common_recommendations_validation,
 product_stats_validation) = read_data(reader, 201501, nb_months_validation, get_profiles)

logging.debug("-- common_recommendations_validation : %s " % len(common_recommendations_validation))
logging.debug("-- personal_recommendations_validation : %s " % len(personal_recommendations_validation))
logging.debug("-- product_stats_validation : %s " % len(product_stats_validation))

personal_recommendations = deepcopy(personal_recommendations_validation)
common_recommendations = deepcopy(common_recommendations_validation)
product_stats = deepcopy(product_stats_validation)

(personal_recommendations,
 common_recommendations,
 product_stats,
 validation_data) = read_data(reader, 201502, 1, get_profiles,
                              return_raw_data=True,
                              personal_recommendations=personal_recommendations,
                              common_recommendations=common_recommendations,
                              product_stats=product_stats)

logging.debug("-- common_recommendations : %s " % len(common_recommendations))
logging.debug("-- personal_recommendations : %s " % len(personal_recommendations))
logging.debug("-- product_stats : %s " % len(product_stats))

reader.close()

In [ ]:
#personal_recommendations_to_proba(personal_recommendations_validation, nb_months_validation+1)

In [ ]:
print('Hashes num: ', len(common_recommendations))
print('Hashes valid num: ', len(common_recommendations_validation))
print('Valid part: ', len(validation_data))

In [ ]:
PERSONAL_RECOMMENDATIONS_WEIGHT = 0.0
COMMON_RECOMMENDATIONS_WEIGHT = 1.0 - PERSONAL_RECOMMENDATIONS_WEIGHT

ZFTURBO_COMMON_WEIGHT = 1.0
MINE_COMMON_WEIGHT = 1.0 - ZFTURBO_COMMON_WEIGHT

In [ ]:
from zfturbo_script_mass_hashes_personal_recommendations import predict_score

In [ ]:
logging.info("- Validation")
map7 = predict_score(validation_data,
                     get_profiles,
                     personal_recommendations_validation,
                     common_recommendations_validation,
                     product_stats_validation,
                     PERSONAL_RECOMMENDATIONS_WEIGHT)

logging.info("Predicted score: {}".format(map7))

## Compare predictions

In [ ]:
from zfturbo_script_mass_hashes_personal_recommendations import apk, get_real_values, compute_predictions
from common import clean_data, process_data

In [ ]:
map71 = 0.0
map72 = 0.0
print('Validation...')
count = 1

personal_recommendations_weight = 0.0

res1 = []
res2 = []

for row in valid_part:

    user = row[1]
    
    predicted2 = zf_get_predictions(arr1, best_valid, overallbest_valid, customer_valid)
    res2.append((user, predicted2))
    real2 = zf_get_real_values(arr1, customer_valid)
    score2 = zf_apk(real2, predicted2)
    map72 += score2
    
    print "ROW : ", row
    prow = clean_data(row)
    if len(prow) > 0:
        prow = process_data(prow)
        predicted1 = compute_predictions(prow, get_profiles,
                                    personal_recommendations_validation,
                                    common_recommendations_validation,
                                    product_stats_validation,
                                    personal_recommendations_weight)

        real1 = get_real_values(prow, personal_recommendations_validation)
        score1 = apk(real1, predicted1)
        map71 += score
        res1.append((user, predicted1))
        
        
    
    count-= 1
    if count == 0:
        break
    
if len(valid_part) > 0:
    map71 /= len(valid_part)
    map72 /= len(valid_part)
    
    
print('Predicted score: {}, {}'.format(map71, map72))    

In [ ]:
print res1[:10]
print res2[:10]

## Compare prediction structures

In [ ]:
common_recommendations_validation.keys()[:10]

In [ ]:
best_valid.keys()[:10]

In [ ]:
import pandas as pd

In [ ]:
df1 = pd.DataFrame(best_valid.keys(), columns=['key', 'country', 'gender', 'age', 'type', 'id', 'mode', '7', '8'])

In [ ]:
df2 = pd.DataFrame(common_recommendations_validation.keys(), columns=['key', 'country', 'gender', 'age', 'type', 'id', 'mode', '7', '8'])

In [ ]:
df2.head()

In [ ]:
df1['key'].value_counts()

In [ ]:
df2['key'].value_counts()

In [ ]:
set(df2[df2['key'] == 3]['id']) - set(df1[df1['key'] == 3]['id'])